### **Modelo de Optimización Logística con Gestión de Combustible- caso 2**

#### **Fundamentos del Modelo**
El desarrollo de este sistema parte de la necesidad de resolver problemas logísticos complejos que integran múltiples restricciones operativas. Se implementó un modelo matemático basado en programación lineal entera mixta (MILP) que considera simultáneamente:

1. **Eficiencia en rutas**: Minimización de distancias recorridas
2. **Economía operacional**: Optimización de costos variables
3. **Gestión de recursos**: Administración de combustible y capacidad vehicular
4. **Factores externos**: Incorporación de peajes y precios diferenciados de combustible

#### **Estructura del Modelo**
El núcleo del sistema se compone de:

**Conjuntos Fundamentales:**
- **Vehículos (V)**: Unidades disponibles con características técnicas específicas
- **Nodos (N)**: Puntos geográficos en la red logística
- **Arcos (A)**: Conexiones válidas entre nodos
- **Estaciones (E)**: Puntos de abastecimiento de combustible
- **Destinos (D)**: Ubicaciones de clientes con demandas específicas

**Parámetros Clave:**
- **Matriz de distancias**: Calculada mediante fórmula haversine para precisión geográfica
- **Consumo y autonomía**: Valores técnicos por tipo de vehículo
- **Estructura de costos**: Combina componentes fijos y variables
- **Precios de combustible**: Diferenciados por estación de servicio

#### **Mecanismos de Optimización**
La función objetivo minimiza el costo total operacional, considerando:
1. Costos de transporte proporcionales a distancia recorrida
2. Gastos en combustible según estación de recarga seleccionada
3. Peajes variables en función de carga transportada

Se implementaron restricciones especializadas para:
- **Conservación de flujo**: Garantiza continuidad en las rutas
- **Prevención de subtours**: Elimina ciclos no conectados al depósito
- **Gestión de combustible**: Asegura autonomía suficiente en cada tramo
- **Satisfacción de demanda**: Cumple con requerimientos de clientes

#### **Innovaciones Implementadas**
1. **Sistema de Recarga Dinámica**:
   - Modela decisiones óptimas de abastecimiento
   - Considera variabilidad en precios de combustible
   - Permite comparar estrategias de recarga completa vs parcial

2. **Integración de Factores Externos**:
   - Incorpora estructura de peajes variables
   - Considera limitaciones de peso en ciertas rutas
   - Modela restricciones operativas realistas

3. **Mecanismo de Linealización**:
   - Transforma relaciones no lineales en formulaciones lineales
   - Permite uso de solvers estándar
   - Mantiene precisión en los cálculos

#### **Proceso de Solución**
El sistema implementa un flujo de trabajo robusto:

1. **Preprocesamiento**:
   - Carga y validación de datos de entrada
   - Cálculo de matriz de distancias
   - Estructuración de conjuntos y parámetros

2. **Optimización**:
   - Configuración del solver con parámetros especializados
   - Manejo de relajaciones y cortes
   - Control de tiempo y calidad de solución

3. **Postprocesamiento**:
   - Validación de factibilidad
   - Generación de reportes detallados
   - Visualización geoespacial de resultados

#### **Ventajas del Enfoque**
1. **Precisión Matemática**: Formulación rigurosa que garantiza optimalidad
2. **Flexibilidad Operativa**: Adaptable a diferentes escenarios logísticos
3. **Eficiencia Computacional**: Tiempos de solución adecuados para problemas reales
4. **Transparencia**: Resultados auditables y explicables

#### **Aplicaciones Prácticas**
El modelo desarrollado permite:
- Planificación estratégica de flotas
- Análisis de costos operacionales
- Evaluación de escenarios alternativos
- Optimización de redes logísticas
- Toma de decisiones basada en datos
.

In [1]:
import sys
!pip install ace-tools
if "google.colab" in sys.modules:

    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"

    import helper

    # Instalar las dependencias necesarias
    helper.install_idaes()
    helper.install_glpk()
    helper.install_ipopt()


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

clients_df=pd.read_csv("clients_caso2.csv")
depots_df=pd.read_csv("depots_caso2.csv")
vehicles_df=pd.read_csv("vehicles_caso2.csv")
stations_df=pd.read_csv("stations_caso2.csv")


In [3]:
from pyomo.environ import *

def build_logistico_model(data):
    model = ConcreteModel()

    # --- Conjuntos ---
    model.V = Set(initialize=data['vehicles'])
    model.N = Set(initialize=data['nodes'])
    model.A = Set(initialize=data['arcs'], dimen=2)
    model.P = Set(initialize=data['ports'])
    model.E = Set(initialize=data['stations'])
    model.D = Set(initialize=data['destinations'])

    # --- Parámetros ---
    model.d  = Param(model.A, initialize=data['distances'])
    model.Cm = Param(initialize=data['Cm'])
    model.Ft = Param(initialize=data['Ft'])
    model.e  = Param(model.V, initialize=data['consumption'])
    model.a  = Param(model.V, initialize=data['autonomy'])
    model.Q  = Param(model.V, initialize=data['capacity'])
    model.q  = Param(model.D, initialize=data['demand'])
    model.W  = Param(model.D, initialize=data['weight_limit'])
    model.p  = Param(model.E, initialize=data['fuel_price'])
    model.Tb = Param(model.A, initialize=data['toll_base'])
    model.Tv = Param(model.A, initialize=data['toll_var'])
    model.M  = Param(initialize=data['BigM'])

    # --- Variables ---
    model.x         = Var(model.V, model.A, domain=Binary)
    model.b         = Var(model.V, model.N, domain=NonNegativeReals)
    model.y         = Var(model.V, model.E, domain=NonNegativeReals)
    model.delivered = Var(model.V, model.D, domain=NonNegativeReals)
    model.ux        = Var(model.V, model.A, domain=NonNegativeReals)
    model.ut = Var(model.V, model.N, domain=Integers,  bounds =(1 , len(model.N)-1) )

    # --- Función Objetivo ---
    def obj_rule(m):
        return sum(
            m.d[i,j] * (m.Cm + m.Ft) * m.x[v,(i,j)]
          + (m.d[i,j] * m.e[v] * (m.p[j] if j in m.E else 0)) * m.x[v,(i,j)]
          + (m.Tb[i,j] + (m.Tv[i,j] * m.delivered[v,j] if j in m.D else 0)) * m.x[v,(i,j)]
            for v in m.V for (i,j) in m.A
        )
    model.obj = Objective(rule=obj_rule, sense=minimize)

    # --- Restricciones de flujo ---
    model.depart    = Constraint(model.V, rule=lambda m,v:
                      sum(m.x[v,(p,j)] for p in m.P for j in m.N if p!=j) <= 1)

    model.flow      = Constraint(model.V, model.N, rule=lambda m,v,j:
                      (sum(m.x[v,(i,j)] for i in m.N if i!=j)
                     == sum(m.x[v,(j,k)] for k in m.N if j!=k))
                      if j not in m.P else Constraint.Skip)

    #model.no_return = Constraint(model.V, model.P, rule=lambda m,v,p:
    #                  sum(m.x[v,(j,p)] for j in m.N if (j,p) in m.A) == 0)

    n=len(model.N)-1
    def mtz_rule ( model ,k, i,j ):
      #i,j = a
      if i != j and i != 1 and j != 1:
        return model.ut[k, i] - model.ut[k,j] + n*model.x[k, (i ,j)] <= n - 1
      return Constraint.Skip


    model.mtz_rules = Constraint(model.V, model.A, rule =  mtz_rule)


    # --- Demanda y capacidad ---
    model.cap    = Constraint(model.V, rule=lambda m,v:
                      sum(m.delivered[v,j] for j in m.D) <= m.Q[v])
    model.demand = Constraint(model.D, rule=lambda m,j:
                      sum(m.delivered[v,j] for v in m.V) == m.q[j])

    # --- Solo entrega si visita ---
    model.link_delivery = Constraint(model.V, model.D, rule=lambda m,v,j:
                              m.delivered[v,j]
                            <= m.Q[v] * sum(m.x[v,(i,j)] for i in m.N if (i,j) in m.A))
#
    # --- Combustible y recarga ---
    model.init_fuel   = Constraint(model.V, rule=lambda m,v:
                              m.b[v,list(m.P)[0]] == m.a[v] * m.e[v])
    model.fuel_update = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                              m.b[v,j] <= m.b[v,i]
                                         - m.d[i,j]*m.e[v]
                                         + (m.y[v,j] if j in m.E else 0)
                                         + m.M*(1-m.x[v,(i,j)]))
    model.fuel_suff   = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                              m.b[v,i] >= m.d[i,j]*m.e[v]
                                         - m.M*(1-m.x[v,(i,j)]))
    model.recharge    = Constraint(model.V, model.E, rule=lambda m,v,j:
                              m.y[v,j] <= m.M * sum(m.x[v,(i,j)] for i in m.N if (i,j) in m.A))

    # --- Linealización de entrega ---
    model.lin1 = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                             m.ux[v,(i,j)] <= m.Q[v] * m.x[v,(i,j)])
    model.lin2 = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                             m.ux[v,(i,j)] <= (m.delivered[v,j] if j in m.D else 0))
    model.lin3 = Constraint(model.V, model.A, rule=lambda m,v,i,j:
                             m.ux[v,(i,j)] >= (m.delivered[v,j] if j in m.D else 0)
                                               - m.Q[v]*(1-m.x[v,(i,j)]))

    return model


In [4]:
import pandas as pd
import math
from pyomo.environ import SolverFactory
#from google.colab import drive

# Coordenadas de municipios
city_coords = {
    'Bogotá':       (4.60971,  -74.08175),
    'Medellín':     (6.25184,  -75.56359),
    'Cali':         (3.43722,  -76.52250),
    'Cartagena':    (10.39972, -75.51444),
    'Cúcuta':       (7.89391,  -72.50782),
    'Bucaramanga':  (7.12539,  -73.11980),
    'Pereira':      (4.81333,  -75.69611),
    'Santa Marta':  (11.24079, -74.19904),
    'Ibagué':       (4.43889,  -75.23222),
    'Manizales':    (5.06889,  -75.51738),
    'Neiva':        (2.92730,  -75.28189),
    'Barranquilla': (10.96854, -74.78132),
    'Villavicencio':(4.14200,  -73.62664),
    'Armenia':      (4.53389,  -75.68111),
}

# Mapear lat/lon a clients_df
clients_df['Latitude']  = clients_df['City/Municipality'].map(lambda c: city_coords[c][0])
clients_df['Longitude'] = clients_df['City/Municipality'].map(lambda c: city_coords[c][1])

# Construir diccionario de ubicaciones
locations = {}
for _, r in depots_df.iterrows():
    locations[r['DepotID']] = (r['Latitude'], r['Longitude'])
for _, r in stations_df.iterrows():
    locations[r['EstationID']] = (r['Latitude'], r['Longitude'])
for _, r in clients_df.iterrows():
    locations[r['LocationID']] = (r['Latitude'], r['Longitude'])

# Definir nodos y arcos
nodes = list(locations.keys())
arcs = [(int(i), int(j)) for i in nodes for j in nodes if i != j]

# Calcular distancias Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.asin(math.sqrt(a))

distances = {(i, j): haversine(*locations[i], *locations[j]) for (i, j) in arcs}

# Preparar data para el modelo
data = {
    'vehicles':     vehicles_df['VehicleID'].tolist(),
    'nodes':        nodes,
    'arcs':         arcs,
    'ports':        depots_df['DepotID'].tolist(),
    'stations':     stations_df['EstationID'].tolist(),
    'destinations': clients_df['LocationID'].tolist(),
    'distances':    distances,
    'Cm':           700,
    'Ft':           5000,
    'consumption':  {vid: 0.2 for vid in vehicles_df['VehicleID']},
    'autonomy':     dict(zip(vehicles_df['VehicleID'], vehicles_df['Range'])),
    'capacity':     dict(zip(vehicles_df['VehicleID'], vehicles_df['Capacity'])),
    'demand':       dict(zip(clients_df['LocationID'], clients_df['Demand'])),
    'weight_limit': {did: float('inf') for did in clients_df['LocationID']},
    'fuel_price':   dict(zip(stations_df['EstationID'], stations_df['FuelCost'])),
    'toll_base':    {(i,j): 0 for (i,j) in arcs},
    'toll_var':     {(i,j): 0 for (i,j) in arcs},
    'BigM':         1e5
}

# Resolver el modelo
model = build_logistico_model(data)
solver = SolverFactory('appsi_highs')
solver.options['parallel'] = 'on'
solver.options['time_limit'] = 20 * 60
solver.options['presolve'] = 'on'
solver.solve(model, tee=True)

# Comprobar que demandas están satisfechas
print("Demandas vs. entregas:")
for j in data['destinations']:
    delivered = sum(model.delivered[v,j].value for v in data['vehicles'])
    print(f"  Nodo {j}: demanda={data['demand'][j]}, entregado={delivered}")


Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
RUN!
MIP  has 6389 rows; 2375 cols; 20155 nonzeros; 1120 integer variables (1050 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+05]
  Cost   [1e+04, 8e+06]
  Bound  [1e+00, 1e+01]
  RHS    [1e+00, 1e+05]
Presolving model
6171 rows, 2298 cols, 19578 nonzeros  0s
6101 rows, 2298 cols, 19508 nonzeros  0s

Solving MIP model with:
   6101 rows
   2298 cols (1049 binary, 70 integer, 0 implied int., 1179 continuous)
   19508 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       

In [5]:
for k in model.V:
  for i in model.A:
    if value(model.x[k,i])>=0.5:
      print(i, "es un nodo elegido para el vehiculo", k)


(1, 15) es un nodo elegido para el vehiculo 1
(3, 6) es un nodo elegido para el vehiculo 1
(4, 3) es un nodo elegido para el vehiculo 1
(6, 7) es un nodo elegido para el vehiculo 1
(7, 13) es un nodo elegido para el vehiculo 1
(10, 12) es un nodo elegido para el vehiculo 1
(12, 4) es un nodo elegido para el vehiculo 1
(13, 1) es un nodo elegido para el vehiculo 1
(15, 10) es un nodo elegido para el vehiculo 1
(1, 5) es un nodo elegido para el vehiculo 2
(3, 8) es un nodo elegido para el vehiculo 2
(5, 3) es un nodo elegido para el vehiculo 2
(8, 11) es un nodo elegido para el vehiculo 2
(11, 1) es un nodo elegido para el vehiculo 2
(1, 15) es un nodo elegido para el vehiculo 3
(2, 13) es un nodo elegido para el vehiculo 3
(13, 1) es un nodo elegido para el vehiculo 3
(14, 2) es un nodo elegido para el vehiculo 3
(15, 14) es un nodo elegido para el vehiculo 3
(1, 9) es un nodo elegido para el vehiculo 4
(9, 13) es un nodo elegido para el vehiculo 4
(13, 1) es un nodo elegido para el veh

In [6]:
import folium
from folium import PolyLine, Marker, CircleMarker, Icon
from IPython.display import display

# 0) Normalizamos los arcos a tuplas de int
arcs_int = [(int(i), int(j)) for (i,j) in data['arcs']]

# 1) Creamos un mapa único, centrado en todas las ubicaciones
center = [
    sum(lat for lat,lon in locations.values()) / len(locations),
    sum(lon for lat,lon in locations.values()) / len(locations)
]
m_all = folium.Map(location=center, zoom_start=6)

# Colores distintos para cada vehículo
colors = ['blue','green','purple','orange','darkred','cadetblue','darkgreen']
# Si tienes más de len(colors) vehículos, los colores se irán repitiendo.

for idx, v in enumerate(data['vehicles']):
    color = colors[idx % len(colors)]
    usados = [(i,j) for (i,j) in arcs_int if model.x[v,(i,j)].value > 0.5]
    print(f"Vehículo {v} tramos activos:", usados)

    # 2) Dibujar cada tramo con el color asignado
    for (i,j) in usados:
        PolyLine(
            [locations[i], locations[j]],
            color=color, weight=3, opacity=0.8,
            popup=f'V{v}: {i}→{j}'
        ).add_to(m_all)

    # 3) Marcar nodos visitados: combustible y recargas
    nodos = set([i for i,j in usados] + [j for i,j in usados])
    for n in nodos:
        lat, lon = locations[n]
        # marcador nivel de combustible
        try:
            combustible = f"Fuel {model.b[v,n].value:.1f} gal"
        except :
            combustible = f"Fuel ignorado"
        CircleMarker(
            (lat, lon), radius=5,
            color=color, fill=True, fill_opacity=0.7,
            popup=f'V{v} Nodo {n}\n {combustible}'
        ).add_to(m_all)
        # marcador de recarga

        try:
            recarga = f"Recarga {model.y[v,n].value:.1f} gal"
            y=model.y[v,n].value
        except:
            recarga = f"Recarga ignorada"
            y=0

        if n in data['stations'] and y:
            Marker(
                (lat, lon),
                icon=Icon(color='lightblue', icon='tint'),
                popup=f'V{v}\n {recarga}'
            ).add_to(m_all)

    # 4) Dibujar peajes en esos tramos
    for (i,j) in usados:
        tb, tv = data['toll_base'][(i,j)], data['toll_var'][(i,j)]
        if tb > 0 or tv > 0:
            PolyLine(
                [locations[i], locations[j]],
                color='red', weight=2, dash_array='5,5',
                popup=f'Peaje {i}→{j}\nBase: {tb} COP\nVar: {tv} COP/ton'
            ).add_to(m_all)

# 5) Finalmente, mostramos el mapa completo
display(m_all)


Vehículo 1 tramos activos: [(1, 15), (3, 6), (4, 3), (6, 7), (7, 13), (10, 12), (12, 4), (13, 1), (15, 10)]
Vehículo 2 tramos activos: [(1, 5), (3, 8), (5, 3), (8, 11), (11, 1)]
Vehículo 3 tramos activos: [(1, 15), (2, 13), (13, 1), (14, 2), (15, 14)]
Vehículo 4 tramos activos: [(1, 9), (9, 13), (13, 1)]
Vehículo 5 tramos activos: []


In [ ]:
import csv
from pyomo.environ import value
from collections import defaultdict

output_filename = "verificacion_caso2_pivot.csv"

datos_por_vehiculo = defaultdict(lambda: defaultdict(list))
totales = defaultdict(lambda: defaultdict(float))

def agregar_dato(v, tipo, origen, destino, val):
    if abs(val) < 1e-3:
        return
    val_redondeado = round(val, 2)

    if tipo == "Ruta (x)":
        # Concatenar como "origen→destino"
        datos_por_vehiculo[v][tipo].append(f"{origen}→{destino}")
        totales[v][tipo] += val  # val siempre 1 para rutas
    elif tipo == "Entrega (delivered)" or tipo == "Recarga (y)" or tipo == "Combustible (b)" or tipo == "Orden (ut)":
        # Concatenar como "destino=valor"
        datos_por_vehiculo[v][tipo].append(f"{destino}={val_redondeado}")
        totales[v][tipo] += val
    elif tipo == "UX (ux)":
        # Concatenar como "origen→destino=valor"
        datos_por_vehiculo[v][tipo].append(f"{origen}→{destino}={val_redondeado}")
        totales[v][tipo] += val


for v in model.V:
    for (i, j) in model.A:
        if value(model.x[v, (i, j)]) >= 0.5:
            agregar_dato(v, "Ruta (x)", i, j, 1)

for v in model.V:
    for j in model.D:
        val = value(model.delivered[v, j])
        agregar_dato(v, "Entrega (delivered)", "", j, val)

for v in model.V:
    for j in model.E:
        val = value(model.y[v, j])
        agregar_dato(v, "Recarga (y)", "", j, val)

for v in model.V:
    for n in model.N:
        val = value(model.b[v, n])
        agregar_dato(v, "Combustible (b)", "", n, val)

for v in model.V:
    for n in model.N:
        if model.ut[v, n].stale or model.ut[v, n].value is None:
            continue
        val = value(model.ut[v, n])
        agregar_dato(v, "Orden (ut)", "", n, val)

for v in model.V:
    for (i, j) in model.A:
        val = value(model.ux[v, (i, j)])
        agregar_dato(v, "UX (ux)", i, j, val)


fieldnames = [
    "Vehiculo",
    "Rutas (origen→destino)",
    "Entregas (destino=valor)",
    "Recargas (destino=valor)",
    "Combustible (destino=valor)",
    "Ordenes (destino=valor)",
    "UX (origen→destino=valor)",
    "Total Ruta (x)",
    "Total Entrega (delivered)",
    "Total Recarga (y)",
    "Total Combustible (b)",
]

rows = []

for v in sorted(model.V):
    fila = {
        "Vehiculo": v,
        "Rutas (origen→destino)": "; ".join(sorted(datos_por_vehiculo[v].get("Ruta (x)", []))),
        "Entregas (destino=valor)": "; ".join(sorted(datos_por_vehiculo[v].get("Entrega (delivered)", []))),
        "Recargas (destino=valor)": "; ".join(sorted(datos_por_vehiculo[v].get("Recarga (y)", []))),
        "Combustible (destino=valor)": "; ".join(sorted(datos_por_vehiculo[v].get("Combustible (b)", []))),
        "Ordenes (destino=valor)": "; ".join(sorted(datos_por_vehiculo[v].get("Orden (ut)", []))),
        "UX (origen→destino=valor)": "; ".join(sorted(datos_por_vehiculo[v].get("UX (ux)", []))),
        "Total Ruta (x)": round(totales[v].get("Ruta (x)", 0), 2),
        "Total Entrega (delivered)": round(totales[v].get("Entrega (delivered)", 0), 2),
        "Total Recarga (y)": round(totales[v].get("Recarga (y)", 0), 2),
        "Total Combustible (b)": round(totales[v].get("Combustible (b)", 0), 2),
    }
    rows.append(fila)

with open(output_filename, mode='w', newline='', encoding='utf-8') as f:

    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print(f"\n Resultados pivotados exportados a: {output_filename}")


✅ Resultados pivotados exportados a: verificacion_caso2_pivot.csv
